In [1]:
import os
path = os.chdir("/Users/sulhasan/Desktop/Project/WRP/src/rephetio")
os.getcwd()

'/Users/sulhasan/Desktop/Project/WRP/src/rephetio'

In [2]:
import os
import sys

import hashlib
import argparse

import numpy as np
import pandas as pd

from tqdm import tqdm
from glmnet import LogitNet # Need to have gnu fortran to work (same for .03)
from itertools import product # didn't go past itertools
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score

import hetnet_ml.graph_tools as gt # remove .src from all
from hetnet_ml.extractor import MatrixFormattedGraph # see line 233 (package from which I'm importing that class)
from hetnet_ml.processing import DegreeTransform, DWPCTransform

ModuleNotFoundError: No module named 'glmnet'

In [9]:
## Set arguments to run the script
parser = argparse.ArgumentParser(description='Run Machine Learning on Time-Based Wikidata Network')
parser.add_argument('data_dir', help="The directory of the source files for machine learning", type=str) # looking for directories within out, not .csv files (because there are multiple files: both nodes and edges)
parser.add_argument('-g', '--gs_treat', help='Replace the TREATS edges in the network with those from the Gold Standard',
                    action='store_true')
parser.add_argument('-a', '--alpha', help="Set the alpha value for the ElasticNet Regression", type=float, default=0.1)
parser.add_argument('-w', '--weight', help="Set the damping exponent for DWPC extraction", type=float, default=0.4)
parser.add_argument('-m', '--multiplier', help="Multiplier for selecting the number of negatives for training.  Will"+
                        " use this factor of the number of positives", type=int, default=10)
parser.add_argument('-s', '--scoring', help='Scoring metric to use for ElasticNet regression', type=str,
                    default='recall')
parser.add_argument('-n', '--num_folds', help='The number of folds for the Cross Validation', type=int, default=5)
parser.add_argument('-e', '--seed', help='Seed for random split between folds', type=int, default=0)
parser.add_argument('-d', '--degree_features', help='Use Degree Features in the model', action='store_true')
parser.add_argument('-c', '--comp_xval', help='Use a Cross-Validation method where holdouts are based on drugs rather'+
                        ' than indication', action='store_true')
parser.add_argument('-l', '--max_length', help='The maximum Lenth for a metapath to be extracted', type=int, default=4)
parser.add_argument('-r', '--remove_similarity_mps', help='Remove Metapaths with CxXxCtD and CtDxXxD pattern.',
                    action='store_true')
parser.add_argument('-f', '--fold_number', help="Run only 1 fold of a 5-fold CV", type=int, default=None)
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [-g] [-a ALPHA] [-w WEIGHT]
                             [-m MULTIPLIER] [-s SCORING] [-n NUM_FOLDS]
                             [-e SEED] [-d] [-c] [-l MAX_LENGTH] [-r]
                             [-f FOLD_NUMBER]
                             data_dir
ipykernel_launcher.py: error: argument -f/--fold_number: invalid int value: '/Users/sulhasan/Library/Jupyter/runtime/kernel-097e8af6-b3c8-47db-8b20-800ff64e4784.json'
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/sulhasan/opt/anaconda3/lib/python3.8/argparse.py", line 2422, in _get_value
    result = type_func(arg_string)
ValueError: invalid literal for int() with base 10: '/Users/sulhasan/Library/Jupyter/runtime/kernel-097e8af6-b3c8-47db-8b20-800ff64e4784.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/sulhasan/opt/anaconda3/lib/python3.8/argparse.py", line 1800, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/Users/sulhasan/opt/anaconda3/lib/python3.8/argparse.py", line 2006, in _parse_known_args
    start_index = consume_optional(start_index)
  File "/Users/sulhasan/opt/anaconda3/lib/python3.8/argparse.py", line 1946, in consume_optional
    take_action(action, args, option_string)
  File "/Users/sulhasan/opt/anaconda3/lib/python3.8/argparse.py", line 1858, in take_action
    argument_values = self._get_values(action, argum

TypeError: object of type 'NoneType' has no len()

In [8]:
nodes = gt.remove_colons(pd.read_csv(os.path.join(data_dir, 'nodes.csv'))) # 'nodes.csv' is a string
edges = gt.remove_colons(pd.read_csv(os.path.join(data_dir, 'edges.csv'))) # edges is a variable

nodes.head()
edges.head()

NameError: name 'data_dir' is not defined

In [ ]:
## Define variables that will govern the network analysis
#remove_mps = ['CrCRrCtD', 'CtDmsMSmsD']
remove_mps = []
#remove_edges = ['CtD']
remove_edges = []
# Test params will be prepended to any output files if they differ from defaults
test_params = '' #'remove_all_sim'

## Unpack command line variables
target_edge_abv = 'CtD'
data_dir = args.data_dir
gs_treat = args.gs_treat
alpha = args.alpha
w = args.weight
negative_multiplier = args.multiplier
scoring = args.scoring
n_folds = args.num_folds
seed = args.seed
include_degree_features = args.degree_features
comp_xval = args.comp_xval
max_length = args.max_length
remove_similarity_mps = args.remove_similarity_mps
fold_number = args.fold_number
if scoring.lower() == 'none':
    scoring = None

# Convert load dir into an integer for a consistent random seed
ini_seed = int(hashlib.sha1(data_dir.encode()).hexdigest(), 16) % 2**16

# Out dir is based on this filename
out_dir = os.path.join('../2_pipeline', sys.argv[0].split('.')[0], 'out')

# Convert non-default parameters to output directory
for k in sorted(list(vars(args).keys())):
    # piecewise folds don't need to be considered a param
    if k == 'fold_number':
        continue
    if k == 'data_dir':
        dirname = os.path.split(vars(args)[k])
        dirname = dirname[-1] if dirname[-1] else dirname[-2]
        test_params += dirname+'.'
        continue
    v = vars(args)[k]
    if v != parser.get_default(k):
        if type(v) == bool:
            test_params += '{}.'.format(k)
        else:
            test_params += '{}-{}.'.format(k, v)
test_params = test_params.rstrip('.')
print('Non-default testing params: {}'.format(test_params))

n_jobs = 32

# Make sure the save directory exists, if not, make it
try:
    os.stat(os.path.join(out_dir, test_params))
except:
    os.makedirs(os.path.join(out_dir, test_params))

def remove_edges_from_gold_standard(to_remove, gs_edges):
    """
    Remove edges from the gold standard
    """
    remove_pairs = set([(tup.c_id, tup.d_id) for tup in to_remove.itertuples()])
    gs_tups = set([(tup.start_id, tup.end_id) for tup in gs_edges.itertuples()])

    remaining_edges = gs_tups - remove_pairs

    return pd.DataFrame({'start_id': [tup[0] for tup in remaining_edges],
                         'end_id': [tup[1] for tup in remaining_edges],
                         'type': 'TREATS_CtD'})

def add_percentile_column(in_df, group_col, new_col, cdst_col='prediction'):

    grpd = in_df.groupby(group_col)
    predict_dfs = []

    for grp, df1 in grpd:
        df = df1.copy()

        total = df.shape[0]

        df.sort_values(cdst_col, inplace=True)
        order = np.array(df.reset_index(drop=True).index)

        percentile = (order+1) / total
        df[new_col] = percentile

        predict_dfs.append(df)

    return pd.concat(predict_dfs, sort=False)

def find_drug_or_disease_similarity(mg):
    """ Finds paths with CxXxCtD or CtDxXxD pattern..."""
    remove_paths = []

    sk = mg.start_kind
    ek = mg.end_kind

    for mp, info in mg.metapaths.items():
        if info['length'] != 3:
            continue
        else:
            # CxXxCtD pattern:
            if (info['edges'][0].split(' - ')[0] == sk and
                    info['edges'][1].split(' - ')[-1] == sk and
                    info['standard_edge_abbreviations'][2] == 'CtD') \
            or (info['standard_edge_abbreviations'][0] == 'CtD' and # CtDxXxD pattern
                    info['edges'][1].split(' - ')[0] == ek and
                    info['edges'][2].split(' - ')[-1] == ek):

                remove_paths.append(mp)
    return remove_paths

def glmnet_coefs(glmnet_obj, X, f_names):
    """Helper Function to quickly return the model coefs and correspoding fetaure names"""
    l = glmnet_obj.lambda_best_[0]

    coef = glmnet_obj.coef_[0]
    coef = np.insert(coef, 0, glmnet_obj.intercept_)

    names = np.insert(f_names, 0, 'intercept')

    z_intercept = coef[0] + sum(coef[1:] * X.mean(axis=0))
    z_coef = coef[1:] * X.values.std(axis=0)
    z_coef = np.insert(z_coef, 0, z_intercept)

    return pd.DataFrame([names, coef, z_coef]).T.rename(columns={0:'feature', 1:'coef', 2:'zcoef'})

def get_scores(y_true, y_proba):
    return roc_auc_score(y_true, y_proba), average_precision_score(y_true, y_proba)

def print_scores(scores):
    print("ROC_AUC: {0:5.3f}\t Average_Precision: {1:5.3f}".format(scores[0], scores[1]))

def add_probas_to_df(fold_num, curr_proba, cd_df):
    cd_df['probas_{}'.format(fold_num)] = curr_proba
    cd_df = add_percentile_column(cd_df, group_col='c_id', new_col='c_percentile_{}'.format(i), cdst_col='probas_{}'.format(i))
    cd_df = add_percentile_column(cd_df, group_col='d_id', new_col='d_percentile_{}'.format(i), cdst_col='probas_{}'.format(i))
    return cd_df

def write_preds(cd_df, filename):
    cd_df.to_csv(filename, index=False)

def make_coef_df(n_folds, coefs):
    # Merge the Coefficient data
    coef_dfs = list()
    for i in range(n_folds):
        coef_dfs.append(coefs[i].copy())
        coef_dfs[i] = coef_dfs[i].set_index('feature')
        coef_dfs[i].columns = [l+'_{}'.format(i) for l in coef_dfs[i].columns]
    coef_df = pd.concat(coef_dfs, axis=1).reset_index()
    return coef_df

def print_top_coefs(coef_df, fold, num=5):
    col = 'coef_{}'.format(fold)
    print(coef_df.sort_values(col, ascending=False)[['feature', col]].head(num), end='\n\n') # SyntaxError: invalid syntax

def write_coefs(coef_df, filename):
    coef_df.to_csv(filename, index=False)

In [ ]:
nodes = gt.remove_colons(pd.read_csv(os.path.join(data_dir, 'nodes.csv'))) # 'nodes.csv' is a string
edges = gt.remove_colons(pd.read_csv(os.path.join(data_dir, 'edges.csv'))) # edges is a variable

comp_ids = set(nodes.query('label == "Compound"')['id'])
dis_ids = set(nodes.query('label == "Disease"')['id'])

# We will use the TREATS edges within the graph as the training for the model
gs_edges = edges.query('type == "causes"').reset_index(drop=True)